In [1]:
%matplotlib inline


from neuron import h
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import h5py

from cell_inference.utils.plotting.plot_results import plot_lfp_traces, plot_lfp_heatmap
from cell_inference.utils.feature_extractors.SummaryStats2D import calculate_stats, build_lfp_grid
# from stylized_module.base.active_model_synapse_input import *
from cell_inference.cells.stylizedcell import CellTypes
from cell_inference.cells.activecell import ActiveCell
from cell_inference.cells.passivecell import PassiveCell
from cell_inference.cells.simulation import Simulation
from cell_inference.utils.currents.recorder import Recorder
from cell_inference.config import paths, params

cell_type = CellTypes.PASSIVE

h.nrn_load_dll(paths.COMPILED_LIBRARY)
geo_standard = pd.read_csv(paths.GEO_STANDARD,index_col='id')
h.tstop = params.TSTOP
h.dt = params.DT

In [2]:
rng = np.random.default_rng(12345)
#            x, y, z,  alpha, h,   phi
loc_param = [0, 0, 50, 0,     1.0, 0.0]
#           [r_s, l_t, r_t, r_d, r_tu, l_d] -> [6.0, 400.0, 0.5, 0.5, 0.5, 200.0]
geo_param = [6.0,400.0,0.5,0.5,0.5,200.0]

inf_list = ['l_t']
number_samples = 1000

xs = np.full((number_samples,1), 0.) if 'xs' not in inf_list else rng.uniform(low=-50, high=50, size=(number_samples,1))
ys = np.full((number_samples,1), 0.) if 'ys' not in inf_list else rng.uniform(low=-2000, high=5000, size=(number_samples,1))
zs = np.full((number_samples,1), 50.) if 'zs' not in inf_list else rng.uniform(low=-50, high=50, size=(number_samples,1))
alphas = np.full((number_samples,1), 0.) if 'alphas' not in inf_list else rng.uniform(low=-(np.pi / 3), high=(np.pi / 3), size=(number_samples,1))
hs = np.full((number_samples,1), 1.) if 'hs' not in inf_list else rng.uniform(low=-1., high=1., size=(number_samples,1))
phis = np.full((number_samples,1), 0.) if 'phis' not in inf_list else rng.uniform(low=0, high=np.pi, size=(number_samples,1))
loc_param = np.concatenate((xs, ys, zs, alphas, hs, phis), axis=1)

r_s = np.full((number_samples,1), 6.0) if 'r_s' not in inf_list else rng.normal(loc=((np.log(3) + np.log(12)) / 2), scale=((np.log(12) - np.log(3)) / 6), size=(number_samples,1))
l_t = np.full((number_samples,1), 400.0) if 'l_t' not in inf_list else rng.uniform(low=20., high=800., size=(number_samples,1))
r_t = np.full((number_samples,1), 0.5) if 'r_t' not in inf_list else rng.normal(loc=((np.log(0.2) + np.log(1.0)) / 2), scale=((np.log(1.0) - np.log(0.2)) / 4), size=(number_samples,1))
r_d = np.full((number_samples,1), 0.5) if 'r_d' not in inf_list else rng.normal(loc=((np.log(0.2) + np.log(1.0)) / 2), scale=((np.log(1.0) - np.log(0.2)) / 4), size=(number_samples,1))
r_tu = np.full((number_samples,1), 0.5) if 'r_tu' not in inf_list else rng.normal(loc=((np.log(0.2) + np.log(1.0)) / 2), scale=((np.log(1.0) - np.log(0.2)) / 4), size=(number_samples,1))
l_d = np.full((number_samples,1), 200.0) if 'l_d' not in inf_list else rng.normal(loc=((np.log(100) + np.log(300)) / 2), scale=((np.log(300) - np.log(100)) / 4), size=(number_samples,1))
geo_param = np.concatenate((r_s, l_t, r_t, r_d, r_tu, l_d), axis=1)

labels = l_t#np.concatenate((loc_param, geo_param), axis=1)
np.set_printoptions(suppress=True)
print(loc_param.shape)
print(geo_param.shape)
print(labels.shape)

(1000, 6)
(1000, 6)
(1000, 1)


In [3]:
hf = h5py.File(paths.INVIVO_DATA_FILE, 'r')
groundtruth_lfp = np.array(hf.get('data'))
hf.close()

if cell_type == CellTypes.ACTIVE:
    sim = Simulation(geometry = geo_standard, 
                     electrodes = params.ELECTRODE_POSITION, 
                     cell_type = CellTypes.ACTIVE, 
                     loc_param = loc_param, 
                     geo_param = geo_param, 
                     gmax = .5, 
                     scale = 1., 
                     ncell = ncell)
# sim = Simulation(geo_standard,params.ELECTRODE_POSITION,loc_param,geo_param=geo_param,gmax=.001,scale=1000, ncell=ncell)  # gmax 0.001 -0.012
else:
    maxIndx = np.argmax(np.absolute(groundtruth_lfp).max(axis=0))  # find maximum absolute value from averaged traces
    maxTrace = -groundtruth_lfp[params.START_IDX:,maxIndx]
    soma_injection = np.insert(maxTrace,0,0.)
    soma_injection = np.asarray([s * params.SOMA_INJECT_SCALING_FACTOR for s in soma_injection])
    sim = Simulation(geometry = geo_standard,
                     electrodes = params.ELECTRODE_POSITION,
                     cell_type = CellTypes.PASSIVE, 
                     loc_param = loc_param,
                     geo_param = geo_param,
                     soma_injection = soma_injection,
                     ncell = number_samples) 
sec_list = sim.cells[0].all
dend_v = Recorder([sec_list[5](0.1),sec_list[5](0.5),sec_list[5](0.9),sec_list[6](0.5)])

In [4]:
lfp_list = []
sim.run_neuron_sim()
for i in range(number_samples):
    lfp_list.append(sim.get_lfp(i).T)
    lfp_list[-1] /= np.max(np.abs(lfp_list[-1]))
    
t = sim.t()
# print(len(t))
# print(np.stack(lfp_list, axis=-1).shape)

In [5]:
params.IM_Y_DISTANCE[240]
data = np.transpose(np.squeeze(np.stack(lfp_list, axis=-1)[:,240,:]))
print(data.shape)

(1000, 801)


In [6]:
from cell_inference.utils.feature_extractors.helperfunctions import train_regression, build_dataloader_from_numpy
train_loader, test_loader = build_dataloader_from_numpy(input_arr=data, labels_arr=labels, batch_size=32)

(1000, 801)


In [7]:
from cell_inference.utils.feature_extractors.fullyconnectednetwork import FullyConnectedNetwork, ActivationTypes
import torch

model1 = FullyConnectedNetwork(in_features=801, out_features=1)

device = torch.device("cpu") #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model1.to(device)

train_regression(model1, train_loader, test_loader, 50, learning_rate=0.05, device=device)

  4%|▍         | 2/50 [00:00<00:03, 14.08it/s]

Training Loss: 2473670.654296875 Validation Loss: 261442.953125


 28%|██▊       | 14/50 [00:00<00:02, 15.34it/s]

Training Loss: 42252.71807861328 Validation Loss: 22243.795043945312


 48%|████▊     | 24/50 [00:01<00:01, 15.41it/s]

...

 64%|██████▍   | 32/50 [00:02<00:01, 13.60it/s]

Training Loss: 27815.496215820312 Validation Loss: 8253.702514648438


 84%|████████▍ | 42/50 [00:02<00:00, 13.93it/s]

Training Loss: 28000.87744140625 Validation Loss: 8016.5672607421875


100%|██████████| 50/50 [00:03<00:00, 14.61it/s]


In [8]:
from sklearn.metrics import r2_score
x, y = next(iter(test_loader))
model1.eval()
x = x.to(device)
output = model1(x)
output = output.to("cpu")
y = y.to("cpu")
print("R2: {}".format(r2_score(y.cpu().detach().numpy(), output.cpu().detach().numpy())))
# print(output)
# print(y)

R2: 0.9714606345938461
